In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Book1.csv to Book1 (1).csv


In [3]:
df = pd.read_csv('Book1.csv',  index_col=None)

In [4]:
X = df.drop(['Disease'], axis=1)
y = df.Disease

In [5]:
import pickle
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)

tree = DecisionTreeClassifier(random_state=42)

tree.fit(X_train, y_train)

Pkl_Filename = "model_tree.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(tree, file)

In [6]:
!pip install colabcode
!pip install fastapi
from pydantic import BaseModel

class Sick(BaseModel):
    vav: float 
    aav: float 
    wss: float 
    vms: float 
    adia: float 
    class Config:
        schema_extra = {
            "example": {
                "vav": 3.8, 
                "aav": 4.7, 
                "wss": 4.3,
                "vms": 14.1,
                "adia": 83
            }
        }

In [7]:
from fastapi import FastAPI
import pickle

app = FastAPI()

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("model_tree.pkl", "rb"))

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}


@app.post('/predict')
def get_disease_category(data: Sick):
    received = data.dict()
    acousticness = received['vav']
    danceability = received['aav']
    energy = received['wss']
    instrumentalness = received['vms']
    liveness = received['adia']
    pred_name = model.predict([[acousticness, danceability, energy,
                                instrumentalness, liveness]]).tolist()[0]
    if pred_name==0:
      return {'Patient is not suffering from any NCD.'}

    elif pred_name==1:
      return {'Patient is suffering from Diabetes.'}
    
    else:
      return{'Patient is suffering from Glaucoma.'}

In [8]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [9]:
server.run_app(app=app)

INFO:     Started server process [592]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "http://15fb5241b904.ngrok.io" -> "http://localhost:10000"
INFO:     223.225.123.61:0 - "GET / HTTP/1.1" 200 OK
INFO:     223.225.123.61:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     223.225.123.61:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     223.225.123.61:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     223.225.123.61:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [592]
